In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from itertools import permutations
import random

print(tf.__version__)

1.13.1


In [2]:
img_width = 224
img_height = 224


In [3]:
tfrecord_train = 'facenet_train.tfrecord'
tfrecord_test = 'facenet_test.tfrecord'
tfrecord_dir = 'tfrecords'

print(os.getcwd())
if not os.path.exists(tfrecord_dir):
    os.makedirs(tfrecord_dir)

C:\Users\beplu\Desktop\ForJupyter\SimpleFaceNet


In [4]:
# 사람 이름 뽑아내기
pic_path = os.path.join(os.getcwd(), 'picture\\WholeDS')
anc_pos_path = os.path.join(os.getcwd(), 'picture\\MoreThanTwo')
print(os.path.exists(pic_path))
print(os.path.exists(anc_pos_path))

True
True


In [5]:
whole_categories = sorted(os.listdir(pic_path))
anc_pos_categories = sorted(os.listdir(anc_pos_path))

In [6]:
anc_pos_set = []
neg_set = []
for category in anc_pos_categories:
    #print(os.listdir(os.path.join(anc_pos_path, category)))
    rnd_list1 = random.sample(os.listdir(os.path.join(anc_pos_path, category)), 2)
    rnd_list1[0] = os.path.join(anc_pos_path, category, rnd_list1[0])
    rnd_list1[1] = os.path.join(anc_pos_path, category, rnd_list1[1])
    #temp = random.sample(os.listdir(os.path.join(anc_pos_path, category)), 2)
    temp_list = whole_categories.copy()
    temp_list.remove(category)
    ran_list = random.sample(temp_list, 15)
    container_list = []
    for neg_category in ran_list:
        rnd_list = random.sample(os.listdir(os.path.join(pic_path, neg_category)), 1)
        rnd_list[0] = os.path.join(pic_path, neg_category, rnd_list[0])
        container_list.append(rnd_list[0])
    
    #flatten 2D -> 1D
    #container_list = sum(container_list, [])
    #print(container_list)

    anc_pos_set.append(rnd_list1)
    neg_set.append(container_list)

print(len(anc_pos_set))
print(len(neg_set))
print(anc_pos_set[0])
print(neg_set[0])    
    

1680
1680
['C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\MoreThanTwo\\Aaron_Peirsol\\Aaron_Peirsol_0003.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\MoreThanTwo\\Aaron_Peirsol\\Aaron_Peirsol_0001.jpg']
['C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Chea_Sophara\\Chea_Sophara_0001.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Andrew_Bunner\\Andrew_Bunner_0001.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Jaromir_Jagr\\Jaromir_Jagr_0001.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Ed_Book\\Ed_Book_0001.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Elizabeth_Smart\\Elizabeth_Smart_0002.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Markus_Naslund\\Markus_Naslund_0001.jpg', 'C:\\Users\\beplu\\Desktop\\ForJupyter\\SimpleFaceNet\\picture\\WholeDS\\Nelson_Shanks

In [7]:
# tfrecord file 경로 설정
train_tfr_path = os.path.join(os.getcwd(), tfrecord_dir, tfrecord_train)
print(train_tfr_path)

# tfrecord writer
writer_train = tf.python_io.TFRecordWriter(train_tfr_path)

C:\Users\beplu\Desktop\ForJupyter\SimpleFaceNet\tfrecords\facenet_train.tfrecord


In [8]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [12]:
index_list = [i for i in range(len(anc_pos_set))]
index_list = np.random.permutation(index_list)
# training set을 tfrecord로 저장
for index in index_list:
    anchor_path = anc_pos_set[index][0]
    pos_path = anc_pos_set[index][1]
    neg_path_bundle = neg_set[index]
    
    anc_image = Image.open(anchor_path)
    anc_image = anc_image.resize((img_height, img_width))
    anc_image = np.asarray(anc_image)
    
    pos_image = Image.open(pos_path)
    pos_image = pos_image.resize((img_height, img_width))
    pos_image = np.asarray(pos_image)
    
    neg_list = []
    for neg_path in neg_path_bundle:
        neg_image = Image.open(neg_path)
        neg_image = neg_image.resize((img_height, img_width))
        neg_image = np.asarray(neg_image)
        #neg_image_str = neg_image.tobytes()
        neg_list.append(neg_image)
    neg_list = np.asarray(neg_list)
    #print(np.shape(neg_list))
    anc_image_str = anc_image.tobytes()
    pos_image_str = pos_image.tobytes()
    neg_image_str = neg_list.tobytes()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'anc_image' : _bytes_feature(anc_image_str),
        'pos_image' : _bytes_feature(pos_image_str),
        'neg_image_bundle' : _bytes_feature(neg_image_str)
    }))
    
    writer_train.write(example.SerializeToString())
    
    

In [13]:
writer_train.close()